#### Imports

In [1]:
import IPython.display as ipd
# % pylab inline
import os
import pandas as pd
import librosa
import glob 
import librosa.display
import random

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.callbacks import EarlyStopping

from keras import regularizers

from sklearn.preprocessing import LabelEncoder

from datetime import datetime

import os

Using TensorFlow backend.


In [2]:
# pip install librosa

#### Reading the files from our folder and creating a dataframe from it

In [11]:
#list the files
filelist = os.listdir('source') 
#read them into pandas
df_male = pd.DataFrame(filelist)
df_male.head()

,0
0,103-1240-0000.flac
1,103-1240-0001.flac
2,103-1240-0002.flac
3,103-1240-0003.flac
4,103-1240-0004.flac


In [12]:
# Adding the 1 label to the dataframe representing male
# df_male['label']='1'
# df_male.head()

In [13]:
# Renaming the column name to file
df_male = df_male.rename(columns={0:'file'})

In [14]:
df_male.head()

,file
0,103-1240-0000.flac
1,103-1240-0001.flac
2,103-1240-0002.flac
3,103-1240-0003.flac
4,103-1240-0004.flac


In [15]:
# Checking for a file that gets automatically generated and we need to drop 
df_male[df_male['file']=='.DS_Store']

,file


Doing the same for the female folder

In [16]:
# filelist = os.listdir('female') 
# #read them into pandas
# df_female = pd.DataFrame(filelist)

In [17]:
# df_female['label']='0'

In [18]:
# df_female = df_female.rename(columns={0:'file'})

In [19]:
# df_female.head()

In [20]:
# Checking for a file that gets automatically generated and we need to drop 
# df_female[df_female['file']=='.DS_Store']

In [40]:
# Dropping the system file
# df_female.drop(981, inplace=True)

In [21]:
# Resetting the index since we dropped a row
# df_female = df_female.reset_index(drop=True)

Joining both dataframes together 

In [22]:
# df = pd.concat([df_female, df_male], ignore_index=True)

In [24]:
df_male.head()

,file
0,103-1240-0000.flac
1,103-1240-0001.flac
2,103-1240-0002.flac
3,103-1240-0003.flac
4,103-1240-0004.flac


In [25]:
df = df_male.sample(frac=1).reset_index(drop=True)

In [26]:
len(df)

2370

We are going to do an split of train, validation and test with 70% train, 20% validation and 10% for test. We check that the randomized splits have balanced classes

In [27]:
df_train = df[:1600]

In [29]:
# df_train['label'].value_counts(normalize=True)

In [30]:
df_validation = df[1600:1800]

In [31]:
# df_validation['label'].value_counts(normalize=True)

In [32]:
df_test = df[1800:]

In [33]:
# df_test['label'].value_counts(normalize=True)

#### Function to extract the features and label for each sound file by iterating through every row of the dataframe

In [44]:
# Although this function was modified and many parameteres were explored with, most of it
# came from Source 8 (sources in the READ.ME)

def extract_features(files):
    
    # Sets the name to be the path to where the file is in my computer
    file_name = os.path.join(os.path.abspath('source')+'/'+str(files.file))

    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    
    # We add also the classes of each file as a label at the end
#     label = files.label

    return mfccs, chroma, mel, contrast, tonnetz #, label

In [45]:
# Code to start the timer to see how long it takes to extract the features
startTime = datetime.now()

In [46]:
# Applying the function to the train data by accessing each row of the dataframe
features_label = df.apply(extract_features, axis=1)

C:\Users\syed\Anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=831
  n_fft, y.shape[-1]
C:\Users\syed\Anaconda3\lib\site-packages\scipy\sparse\lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
C:\Users\syed\Anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=795
  n_fft, y.shape[-1]
C:\Users\syed\Anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=946
  n_fft, y.shape[-1]
C:\Users\syed\Anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=865
  n_fft, y.shape[-1]
C:\Users\syed\Anaconda3\lib\site-packages\librosa\core\spectrum.py:224: UserWar

In [47]:
# Code to see how long it took
print(datetime.now() - startTime)

0:37:21.053588


In [48]:
# Checking how the output looks
features_label

0       ([-358.4652, 91.62154, -54.416817, 29.428865, ...
1       ([-337.83096, 118.637085, -43.04103, 55.654465...
2       ([-442.1549, 123.60432, 1.0404664, 49.69681, -...
3       ([-430.2393, 119.45682, -21.997358, 39.519283,...
4       ([-368.61032, 123.3883, -40.216217, 54.60901, ...
5       ([-331.19516, 99.19405, -29.1898, 51.928238, -...
6       ([-286.22995, 94.15037, -60.42427, 67.472046, ...
7       ([-334.2393, 118.76558, -35.435295, 51.88353, ...
8       ([-381.3552, 101.365906, -33.492317, 54.846992...
9       ([-233.70636, 131.73514, -22.442453, 35.748188...
10      ([-326.0352, 133.74205, -5.9101596, 51.428097,...
11      ([-283.60043, 95.91734, -60.80046, 69.603745, ...
12      ([-410.60834, 138.47823, -33.196297, 31.917805...
13      ([-323.8894, 115.3959, -52.464832, 58.539745, ...
14      ([-368.02167, 129.70161, -28.490015, 58.119934...
15      ([-363.85992, 113.357895, -41.577885, 27.71254...
16      ([-344.3103, 114.602425, -42.454277, 69.62651,...
17      ([-333

In [49]:
# The next code in markdown saves the numpy array (in case our kernel restarts or 
# anything happens, because it takes long to extract the features)

# np.save('features_label', features_label)

In [50]:
# The next code loads the saved numpy array of our extracted features
# features_label = np.load('features_label.npy', allow_pickle=True)

In [51]:
# We create an empty list where we will concatenate all the features into one long feature
# for each file to feed into our neural network 

features = []
for i in range(0, len(features_label)):
    features.append(np.concatenate((features_label[i][0], features_label[i][1], 
                features_label[i][2], features_label[i][3],
                features_label[i][4]), axis=0))

In [52]:
len(features)

2370

#### We will add the speaker id to our dataframe to have that as the label for our model and predict speakers from their voice

In [53]:
# We create an empty list where we will append all the speakers ids for each row of our
# dataframe by slicing the file name since we know the id is the first numbers before the hash
speaker = []
for i in range(0, len(df)):
    speaker.append(df['file'][i].split('-')[0])

In [54]:
# Now we create the speaker column in our dataframe and set it equal to our speaker list
df['speaker'] = speaker

In [55]:
# Checking that it worked as expected
df.head()

,file,speaker
0,87-121553-0067.flac,87
1,27-124992-0078.flac,27
2,163-122947-0021.flac,163
3,198-126831-0040.flac,198
4,27-123349-0024.flac,27


In [56]:
# Checking the number of speakers or the number of different people in our voice data
df['speaker'].nunique()

20

In [57]:
# Setting our labels to be equal to our speaker list
labels = speaker

In [58]:
# Checking the size of labels and making sure it matches the size of features
len(labels)

2370

#### Checking if we have balanced classes for the whole data

In [59]:
# They look somewhat balanced with a min of 56 and a max of 166, mean of 114 
# with standard deviation of 15.89 (calculated from scipy)
np.unique(labels, return_counts=True)

(array(['103', '118', '125', '150', '163', '19', '196', '198', '200',
        '201', '26', '27', '32', '39', '40', '60', '78', '83', '87', '89'],
       dtype='<U3'),
 array([102, 137, 138, 114, 112, 111, 108, 126, 116, 127, 118, 138, 117,
        123, 114,  97, 118, 123, 108, 123], dtype=int64))

#### Hot encoding y and pre processing X and y

In [60]:
X = np.array(features)

In [61]:
y = np.array(labels)

In [62]:
# Hot encoding y
lb = LabelEncoder()
y = to_categorical(lb.fit_transform(y))

In [63]:
X.shape

(2370, 193)

In [64]:
y.shape

(2370, 20)

In [65]:
# Choosing the first 9188 (70%) files to be our train data
# Choosing the next  2625 (20%) files to be our validation data
# Choosing the next  1312 (10%) files to be our test never before seen data
# This is analogous to a train test split but we add a validation split and we are making
# we do not shuffle anything since we are dealing with several time series, we already 
# checked before that we have balanced classes (analogous to stratify)

X_train = X[:1600]
y_train = y[:1600]

X_val = X[1600:2000]
y_val = y[1600:2000]

X_test = X[2000:]
y_test = y[2000:]

In [66]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [73]:
# Build a simple dense model with early stopping with softmax for categorical classification
# We have 115 classes 
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential()

model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dropout(0.1))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))  

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))    

model.add(Dense(20, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

In [74]:
# fitting the model with the train data and validation with the validation data
# we used early stop with patience 100 because we did not want to use early stop
# I leave the early stop regularization code in case anyone wants to use it

history = model.fit(X_train, y_train, batch_size=256, epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])
                    

Train on 1600 samples, validate on 400 samples
Epoch 1/100
1600/1600 [==============================] - ETA: 5s - loss: 3.2745 - acc: 0.046 - ETA: 1s - loss: 3.1241 - acc: 0.071 - ETA: 0s - loss: 2.9552 - acc: 0.117 - 1s 796us/sample - loss: 2.9480 - acc: 0.1200 - val_loss: 2.3957 - val_acc: 0.6175
Epoch 2/100
1600/1600 [==============================] - ETA: 0s - loss: 2.5667 - acc: 0.273 - 0s 33us/sample - loss: 2.3432 - acc: 0.3681 - val_loss: 1.8140 - val_acc: 0.7325
Epoch 3/100
1600/1600 [==============================] - ETA: 0s - loss: 2.0175 - acc: 0.492 - 0s 36us/sample - loss: 1.8106 - acc: 0.5319 - val_loss: 1.1453 - val_acc: 0.8400
Epoch 4/100
1600/1600 [==============================] - ETA: 0s - loss: 1.5228 - acc: 0.636 - 0s 35us/sample - loss: 1.3232 - acc: 0.6544 - val_loss: 0.6030 - val_acc: 0.9200
Epoch 5/100
1600/1600 [==============================] - ETA: 0s - loss: 1.0707 - acc: 0.710 - 0s 31us/sample - loss: 0.9343 - acc: 0.7400 - val_loss: 0.3124 - val_acc: 0.9

1600/1600 [==============================] - ETA: 0s - loss: 0.0176 - acc: 0.992 - 0s 36us/sample - loss: 0.0120 - acc: 0.9969 - val_loss: 0.0159 - val_acc: 0.9975
Epoch 47/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0042 - acc: 1.000 - 0s 36us/sample - loss: 0.0080 - acc: 0.9981 - val_loss: 0.0162 - val_acc: 0.9975
Epoch 48/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0070 - acc: 1.000 - 0s 27us/sample - loss: 0.0109 - acc: 0.9987 - val_loss: 0.0160 - val_acc: 0.9975
Epoch 49/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0077 - acc: 1.000 - 0s 26us/sample - loss: 0.0103 - acc: 0.9987 - val_loss: 0.0169 - val_acc: 0.9975
Epoch 50/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0064 - acc: 1.000 - 0s 37us/sample - loss: 0.0089 - acc: 0.9987 - val_loss: 0.0173 - val_acc: 0.9975
Epoch 51/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0126 - acc: 1.000 - 0s 30us/sample - loss: 0.0133 - 

Epoch 91/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0044 - acc: 0.996 - 0s 28us/sample - loss: 0.0055 - acc: 0.9981 - val_loss: 0.0170 - val_acc: 0.9975
Epoch 92/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0016 - acc: 1.000 - 0s 26us/sample - loss: 0.0035 - acc: 0.9987 - val_loss: 0.0173 - val_acc: 0.9975
Epoch 93/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0032 - acc: 1.000 - 0s 27us/sample - loss: 0.0052 - acc: 0.9981 - val_loss: 0.0174 - val_acc: 0.9975
Epoch 94/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0034 - acc: 1.000 - 0s 30us/sample - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0169 - val_acc: 0.9975
Epoch 95/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0025 - acc: 1.000 - 0s 36us/sample - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0173 - val_acc: 0.9975
Epoch 96/100
1600/1600 [==============================] - ETA: 0s - loss: 0.0024 - acc: 1.000 - 0s 35us/sample - lo

In [80]:
#history.history[]

In [83]:
# Check out our train accuracy and validation accuracy over epochs.
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
train_accuracy = history.history['acc']
val_accuracy = history.history['val_acc']

# Set figure size.
plt.figure(figsize=(12, 8))

# Generate line plot of training, testing loss over epochs.
plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')

# Set title
plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Categorical Crossentropy', fontsize = 18)
plt.xticks(range(0,100,5), range(0,100,5))

plt.legend(fontsize = 18);

In [84]:
# We get our predictions from the test data
preds = model.predict_classes(X_test)

In [85]:
# We transform back our predictions to the speakers ids
preds = lb.inverse_transform(preds)

In [87]:
# We slice our dataframe to our test dataframe
df_test = df[2000:]

In [88]:
# We create a new column called preds and set it equal to our predictions
df_test['preds'] = preds

C:\Users\syed\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [89]:
# Checking how our test dataframe looks like now with our predictions
df_test

,file,speaker,preds
2000,89-218-0051.flac,89,89
2001,118-47824-0013.flac,118,118
2002,39-121916-0005.flac,39,39
2003,27-124992-0019.flac,27,27
2004,125-121124-0013.flac,125,125
2005,26-496-0022.flac,26,26
2006,87-121553-0083.flac,87,87
2007,150-132655-0005.flac,150,150
2008,78-368-0033.flac,78,78
2009,118-47824-0082.flac,118,118


In [90]:
# Checking how many speakers we got wrong
df_test[df_test['speaker'] != df_test['preds']]

,file,speaker,preds


In [91]:
# Checking our model accuracy
1-round(len(df_test[df_test['speaker'] != df_test['preds']])/len(df_test),3)

1.0

#### 99.8% accurate on test data for classification of speakers